# 1D ResNet Implementation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class SimpleResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SimpleResidualBlock1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()  # Define ReLU as a layer
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else None

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        if self.shortcut:
            identity = self.shortcut(x)
        
        out += identity
        return self.relu(out)


In [11]:
class SimpleResNet1D(nn.Module):
    def __init__(self):
        super(SimpleResNet1D, self).__init__()
        self.initial_conv = nn.Conv1d(1, 16, kernel_size=7, padding=3)
        self.initial_bn = nn.BatchNorm1d(16)
        self.relu = nn.ReLU()
        self.residual_block = SimpleResidualBlock1D(16, 16)
        self.fc = nn.Linear(16, 1)

    def forward(self, x):
        x = self.relu(self.initial_bn(self.initial_conv(x)))
        x = self.residual_block(x)
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)
        return x

Sample input: batch of 10 1D signals with 100 timesteps

In [12]:
sample_input = torch.randn(10, 1, 100)  # (batch_size, channels, length)
model = SimpleResNet1D()
output = model(sample_input)
print("Output shape:", output.shape)  # Should output: [10, 1]


Output shape: torch.Size([10, 1])
